### General rules:
 * For all figures that you generate, remember to add meaningful labels to the axes (including units), and provide a legend and colorbar, if applicable.
 * Do not hard code constants, like number of samples, number of channels, etc in your program. These values should always be determined from the given data. This way, you can easily use the code to analyse other data sets.
 * Do not use high-level functions from toolboxes like scikit-learn.
 * Before submitting, check your code by executing: Kernel -> Restart & run all.
 * Replace *Template* by your *FirstnameLastname* in the filename, or by *Lastname1Lastname2* if you work in pairs.

# BCI-IL - Exercise Sheet #11 BONUS

#### Name:

In [1]:
% matplotlib inline

import numpy as np
import scipy as sp
import scipy.signal
from matplotlib import pyplot as plt

import bci_minitoolbox as bci
import bci_classifiers as cfy

## Preparation: Load data

In [6]:
fname = 'imagVPaw.npz'
cnt, fs, clab, mnt, mrk_pos, mrk_class, mrk_className = bci.load_data(fname)
print(cnt.shape)
print(mrk_pos.shape)

(51, 282838)
(280,)


In [3]:
def train_CSP(epo, mrk_class):
    ''' 
    Usage:
        W, d = trainCSP(epo, mrk_class)
    Parameters:
        epo:   a 3D array of segmented signals (samples x channels x epochs)
        mrk_class: a 1D array that assigns markers to classes (0, 1)
    Returns:
        W:     matrix of spatial filters
        d:     vector of generalized Eigenvalues
    '''
    C = epo.shape[1]
    X1 = np.reshape(np.transpose(epo[:,:,mrk_class==0], (1,0,2)), (C, -1))
    S1 = np.cov(X1)
    X2 = np.reshape(np.transpose(epo[:,:,mrk_class==1], (1,0,2)), (C, -1))
    S2 = np.cov(X2)	
    d, W = scipy.linalg.eigh(a=S1, b=S1+S2)
    return W, d

In [4]:
def crossvalidation(classifier_fcn, X, y, folds=10, verbose=False, feature_extraction=None):
    '''
    Synopsis:
        loss_te, loss_tr= crossvalidation(classifier_fcn, X, y, folds=10, verbose=False)
    Arguments:
        classifier_fcn: handle to function that trains classifier as output w, b
        X:              data matrix (features X epochs) or epochs (samples X channels x epochs) 
                                with feature_extraction producing a data matrix (features X epochs)
        y:              labels with values 0 and 1 (1 x epochs)
        folds:         number of folds
        verbose:        print validation results or not
        feature_extraction: a function producing a data matrix (features X epochs) out of epoched data
    Output:
        loss_te: value of loss function averaged across test data
        loss_tr: value of loss function averaged across training data
    '''
    
    if len(X.shape)==2:
        nDim, nSamples = X.shape
        feature_extraction=lambda X,Y,Z:X
    elif len(X.shape)==3:
        nT,nCh, nSamples = X.shape
        if feature_extraction==None:
            raise TypeError('For epoched data X, a feature extraction function has to be defined!')
        
    inter = np.round(np.linspace(0, nSamples, num=folds + 1)).astype(int)
    perm = np.random.permutation(nSamples)
    errTr = np.zeros([folds, 1])
    errTe = np.zeros([folds, 1])

    for ff in range(folds):
        idxTe = perm[inter[ff]:inter[ff + 1] + 1]
        idxTr = np.setdiff1d(range(nSamples), idxTe)
        fv=feature_extraction(X,y,idxTr)
        w, b = classifier_fcn(fv[:, idxTr], y[idxTr])
        out = w.T.dot(fv) - b
        errTe[ff] = cfy.loss_weighted_error(out[idxTe], y[idxTe])
        errTr[ff] = cfy.loss_weighted_error(out[idxTr], y[idxTr])

    if verbose:
        print('{:5.1f} +/-{:4.1f}  (training:{:5.1f} +/-{:4.1f})  [using {}]'.format(errTe.mean(), errTe.std(),
                                                                                     errTr.mean(), errTr.std(), 
                                                                                     classifier_fcn.__name__))
    
    return np.mean(errTe), np.mean(errTr)

In [5]:
def train_CSPlogvar(epo, mrk_class, idxTr, NumComp = 6):
    ''' 
    Usage:
        W, d = train_CSP_LDA(epo, mrk_class)
    Parameters:
        epo:   a 3D array of segmented signals (samples x channels x epochs)
        mrk_class: a 1D array that assigns markers to classes (0, 1)
        idxTr: indexes of the data to train CSP
    Returns:
        W:     matrix of spatial filters
        d:     vector of generalized Eigenvalues
    '''
    W, d = train_CSP(epo[:,:, idxTr], mrk_class[idxTr])
    selected_csps = np.flipud(np.argsort(np.maximum(d,1-d)))[:NumComp]
    W_csp = W[:, selected_csps]
    epo_csp = np.dot(W_csp.T, epo)
    epo_csp = np.log(np.var(epo_csp,axis=1))
    return epo_csp

## Exercise 1: Validation of CSP-based Classification  (10 BONUS points)
This task is a continuation of the exercises on the sheet \#08 and uses the same dataset. <br>
Classification can be performed using log band-power features of the CSP filtered and segmented signals. Use the frequency-band, time-interval and the same selection of six CSP filters (as charaterized by the index) as on the previous sheet. Estimate the generalization error using the following three different procedures and compare the results:
1. Determine the CSP filters on the first half of the epochs, calculate log band-power features from those epochs, train an LDA classifier and determine the classification error on those features (training error).
2. Determine CSP filters and LDA classifier on the first half of the data as in (1), but determine the classification error on the second half of the data as required for a sound validation.
3. Determine CSP filters on the first half of the data and determine the classification error by cross-validating LDA on the log band-power features.

In [19]:
# Determine the CSP filters on the first half of the epochs, calculate log band-power features from those epochs,
# train an LDA classifier and determine the classification error on those features (training error).

band = [10.5, 13]
ival = [1000, 4500]
Wn = band / fs * 2
b, a = sp.signal.butter(5, Wn, btype='bandpass')
cnt_bp = sp.signal.lfilter(b, a, cnt)
epo, epo_t = bci.makeepochs(cnt_bp, fs, mrk_pos, ival)

halfind= int(mrk_pos.shape[0]/2)

epo_1 = epo[:,:,:halfind]
#print(epo_1.shape)
mrk_class_1 = mrk_class[:halfind]
#print(mrk_class_1.shape)

W, d = train_CSP(epo_1, mrk_class_1)
print(W.shape,d.shape)

selected_csps = np.flipud(np.argsort(np.maximum(d,1-d)))[:6]
print(selected_csps)

W_csp = W[:, selected_csps]
print(W_csp.shape)

epo_csp_1 = np.dot(W_csp.T, epo_1)
print(epo_csp_1.shape)

epo_csp_1 = np.log(np.var(epo_csp_1,axis=1))
print(epo_csp_1.shape)

loss_test_1, loss_train_1 = cfy.crossvalidation(cfy.train_LDA, epo_csp_1, mrk_class_1, verbose = True)

a1, b1 = cfy.train_LDA(epo_csp_1, mrk_class_1)
print(a1,b1)

out_1 = a1.T.dot(epo_csp_1) - b1
print(out_1.shape)

loss_1 = cfy.loss_weighted_error(out_1, mrk_class_1)
print("training error for case 1:", loss_1)


(51, 51) (51,)
[50  0 49 48  1 47]
(51, 6)
(6, 351, 140)
(6, 140)
  1.5 +/- 3.0  (training:  0.8 +/- 0.3)  [using train_LDA]
[-8.36929932  4.47757869  0.29347242 -1.8282518   6.01006481 -1.70828983] 3.409931495873446
(140,)
training error for case 1: 0.7142857142857143


In [20]:
# Determine CSP filters and LDA classifier on the first half of the data as in (1),
# but determine the classification error on the second half of the data as required for a sound validation.

epo_2 = epo[:,:,halfind:]
print(epo_2.shape)
mrk_class_2 = mrk_class[halfind:]

epo_csp_2 = np.dot(W_csp.T, epo_2)
print(epo_csp_2.shape)

epo_csp_2 = np.log(np.var(epo_csp_2,axis=1))
print(epo_csp_2.shape)

#loss_test_2, loss_train_2 = cfy.crossvalidation(cfy.train_LDA, epo_csp_2, mrk_class_2, verbose = True)
a2, b2 = cfy.train_LDA(epo_csp_2, mrk_class_2)
print(a2,b2)

out_2 = a2.T.dot(epo_csp_2) - b2
print(out_1.shape)

#loss_2 = cfy.loss_weighted_error(out_1, mrk_class_2) # was it meant to show error between determined labels from 1st part with true labels from second part??
loss_2 = cfy.loss_weighted_error(out_2, mrk_class_2)
print("training error for case 2:", loss_2)

(351, 51, 140)
(6, 351, 140)
(6, 140)
[-10.12453781   2.7750425    0.09409441   0.39106803   4.34501542
  -0.32733703] 4.649855406556106
(140,)
training error for case 2: 2.142857142857143


In [8]:
# Determine CSP filters on the first half of the data and 
# determine the classification error by cross-validating LDA on the log band-power features.

crossvalidation(cfy.train_LDA, epo_1, mrk_class_1, folds=10, verbose=True, feature_extraction=train_CSPlogvar)

  2.3 +/- 2.8  (training:  0.5 +/- 0.5)  [using train_LDA]


(2.2698412698412698, 0.47771857398873524)